In [77]:
import pandas as pd

In [65]:
df = pd.read_csv('/Users/srivatsasinha/Desktop/assignment/anlp-monsoon-24/dataset/cnn_dailymail/validation.csv')

In [66]:
df.head()

,id,article,highlights
0,61df4979ac5fcc2b71be46ed6fe5a46ce7f071c3,"Sally Forrest, an actress-dancer who graced th...","Sally Forrest, an actress-dancer who graced th..."
1,21c0bd69b7e7df285c3d1b1cf56d4da925980a68,A middle-school teacher in China has inked hun...,Works include pictures of Presidential Palace ...
2,56f340189cd128194b2e7cb8c26bb900e3a848b4,A man convicted of killing the father and sist...,"Iftekhar Murtaza, 29, was convicted a year ago..."
3,00a665151b89a53e5a08a389df8334f4106494c2,Avid rugby fan Prince Harry could barely watch...,Prince Harry in attendance for England's crunc...
4,9f6fbd3c497c4d28879bebebea220884f03eb41a,A Triple M Radio producer has been inundated w...,Nick Slater's colleagues uploaded a picture to...


In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13368 entries, 0 to 13367
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          13368 non-null  object
 1   article     13368 non-null  object
 2   highlights  13368 non-null  object
dtypes: object(3)
memory usage: 313.4+ KB


In [68]:
train_articles = df['article'].to_list()

In [26]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125m")
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125m")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

#print(model)

1

In [69]:
tokenised_train_articles = tokenizer(train_articles)

In [70]:
import numpy as np


# Step 1: Compute lengths of each sublist
lengths = [len(sublist) for sublist in tokenised_train_articles['input_ids']]
percentile = 80
# Step 2: Find the 90th percentile of lengths
percentile_90 = np.percentile(lengths, percentile)

# Step 3: Count how many sequences have length <= 90th percentile
count = sum(1 for length in lengths if length <= percentile_90)

print(f"{percentile}th percentile of sequence lengths: {percentile_90}")
print(f"Number of sequences with length <= {percentile}th percentile: {count}")


80th percentile of sequence lengths: 1177.6000000000004
Number of sequences with length <= 80th percentile: 10694


In [55]:
lengths = [len(sublist) for sublist in tokenised_train_articles['input_ids']]

In [71]:
max(lengths)

2491

In [72]:
# Given length value
given_value = 1000

# Get indices of sublists whose length is less than the given value
indices = [i for i, sublist in enumerate(tokenised_train_articles['input_ids']) if len(sublist) < given_value]

# Display the indices
#print(f"Indices of sublists with length less than {given_value}: {indices}")
print(len(indices))

9319


In [73]:
# Number of indices to sample
N = 9319

# Step 1: Randomly sample N indices from the given list
sampled_indices = np.random.choice(indices, N, replace=False)

# Step 2: Fetch corresponding rows from the DataFrame
sampled_rows = df.iloc[sampled_indices]


In [75]:
sampled_rows.head()

,id,article,highlights
5364,0547f560b663b30b0a61c773a9f4c5b22ef02eca,"Guo Hong, 29, always has at least three or fou...","Guo Hong, 29, started off teaching two classes..."
11374,cac38f286a63f492a0052814e46d9b90fd4be45a,Should a man pay for dinner while on a date wi...,Emma Watson tried to pay for meal on date and ...
2811,82f91be4f41e85592689f6dd4d7d1efc81660d9f,Wallis Simpson ordered a top level diplomatic ...,Wallis Simpson made diplomats fetch her swimsu...
8508,b371218b25641df4e1bfe6f392891bd3626bbdce,"Spencer Robinson, 36 was arrested after he all...","Utah police said Spencer Robinson, 36, told th..."
8862,803c7c71969985b7c479425d04efdb9e6eb89d10,Manchester United manager Louis van Gaal has r...,Louis van Gaal has revealed Manchester United ...


In [41]:
tokenised_train_articles_sampled = tokenizer(sampled_rows['article'].to_list(),padding='max_length',max_length=1000)

In [44]:
len(tokenised_train_articles_sampled['attention_mask'][0])

1000

In [76]:
sampled_rows.to_csv('/Users/srivatsasinha/Desktop/assignment/anlp-monsoon-24/dataset/cnn_dailymail/validation_sampled.csv')